In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
#from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook
import zipfile
import pickle

from torch.utils.data import DataLoader, Dataset

from dataset import *

%matplotlib inline

In [2]:
SEED = 1234
# BATCH_SIZE = 16
# NUM_WORKERS = 20
debug = False#True

In [4]:
%%time
fname_train, fname_valid = train_test_split(SEED, debug)
print(len(fname_train), len(fname_valid))

total trainset:  53535
48412 5123
CPU times: user 8.7 s, sys: 172 ms, total: 8.88 s
Wall time: 8.68 s


In [4]:
x_train, x_valid, y_train, y_valid, fname_train, fname_valid = make_trainset(SEED, debug)

total trainset:  31072


In [3]:
#fname_arr = train_label['Id'].values.reshape(-1, 1)
#fname_train, target_train, fname_valid, target_valid = iterative_stratification(fname_arr, target_arr, SEED)
#fname_train, fname_valid = fname_train[:, 0], fname_valid[:, 0]

train_label = get_label_data(use_external=True)

In [4]:
target_list = []
for i in train_label.index:
    target = multi_hot(train_label.loc[i, 'Target'])
    target_list.append(target)
    
target_arr = np.array(target_list)
#fname_arr = train_label['Id'].values.reshape(-1, 1)

In [5]:
target_arr.sum(axis=0), target_arr.sum()

(array([17977,  3029,  4833,  3263,  5030,  5811,  3647,  3658,   207,
          197,   182,  2173,  2189,  1424,  2650,    57,  1275,   444,
         1871,  3614,   421,  6356,  2664,  3663,   424, 14591,   688,
          127]), 92465)

In [14]:
pd.value_counts(target_arr.sum(axis=1))

1    23540
2    21959
3     7141
4      851
5       36
dtype: int64

In [30]:
from dataset_single_multi_label import get_label_data
from sklearn.model_selection import train_test_split
    
##label
train_label = get_label_data()
#train_label = train_label.sample(n=1000, axis=0, random_state=SEED)
print('total trainset: ', len(train_label.index))

fname_arr = train_label['Id'].values.reshape(-1, 1)
fname_train, fname_valid = train_test_split(fname_arr, test_size=0.1, random_state=1234)

total trainset:  53527


## class-weights and sampler

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook
import pickle

from gpu_utils import set_n_get_device

#import torch
from dataset import *

SEED = 1234
BATCH_SIZE = 32#16
NUM_WORKERS = 20
debug = True#True

device = set_n_get_device("0, 1, 2, 3", data_device_id="cuda:2")#0, 1, 2, 3, IMPORTANT: data_device_id is set to free gpu for storing the model, e.g."cuda:1"
multi_gpu = [2, 3]#[1, 2]#None, 0, 1, 2, 3

class_weights_dict = {0: 1.0, 1: 3.01, 2: 1.95, 3: 2.79, 4: 2.61, 5: 2.31, 6: 3.23, 7: 2.2, 8: 6.17, 9: 6.34, 10: 6.81, 11: 3.15, 12: 3.61, 13: 3.86, 14: 3.17, 15: 7.1, 16: 3.87, 17: 4.8, 18: 3.34, 19: 2.84, 20: 4.99, 21: 1.91, 22: 3.46, 23: 2.15, 24: 4.37, 25: 1.13, 26: 4.35, 27: 7.74}
class_weights_arr = np.array([1.0, 3.01, 1.95, 2.79, 2.61, 2.31, 3.23, 2.2, 6.17, 6.34, 6.81, 3.15, 3.61, 3.86, 3.17, 7.1, 3.87, 4.8, 3.34, 2.84, 4.99, 1.91, 3.46, 2.15, 4.37, 1.13, 4.35, 7.74])
print(class_weights_dict)
print(class_weights_arr)

{0: 1.0, 1: 3.01, 2: 1.95, 3: 2.79, 4: 2.61, 5: 2.31, 6: 3.23, 7: 2.2, 8: 6.17, 9: 6.34, 10: 6.81, 11: 3.15, 12: 3.61, 13: 3.86, 14: 3.17, 15: 7.1, 16: 3.87, 17: 4.8, 18: 3.34, 19: 2.84, 20: 4.99, 21: 1.91, 22: 3.46, 23: 2.15, 24: 4.37, 25: 1.13, 26: 4.35, 27: 7.74}
[1.   3.01 1.95 2.79 2.61 2.31 3.23 2.2  6.17 6.34 6.81 3.15 3.61 3.86
 3.17 7.1  3.87 4.8  3.34 2.84 4.99 1.91 3.46 2.15 4.37 1.13 4.35 7.74]


In [2]:
train_dl, val_dl = prepare_trainset(BATCH_SIZE, NUM_WORKERS, SEED, debug, sample_mode='raw')#raw, balance, weak_balance


total trainset:  1000
Count of trainset:  903
Count of validset:  97
Count of trainset (for training):  900
Count of validset (for training):  100
calculate label class_weights
0.0  min


In [ ]:
s = 0
c = 0

for i, (image, target) in enumerate(tqdm_notebook(train_dl)):
    #print(i)
    image = image.to(device=device, dtype=torch.float)
    target = target.to(device=device, dtype=torch.float)
    s += target.sum(dim=0)
    c += 1
    if i == 30:
        break

In [5]:
# raw without sampler
s

tensor([383.,  38., 109.,  52.,  44.,  70.,  35.,  77.,   2.,   2.,   2.,  37.,
         21.,  13.,  31.,   0.,  20.,  10.,  32.,  34.,  10., 146.,  25.,  82.,
          8., 302.,   6.,   3.], device='cuda:2')

In [11]:
s/c * torch.from_numpy(np.array(class_weights)).to(device=device, dtype=torch.float)

tensor([12.3548,  3.6897,  6.8565,  4.6800,  3.7045,  5.2161,  3.6468,  5.4645,
         0.3981,  0.4090,  0.4394,  3.7597,  2.4455,  1.6187,  3.1700,  0.0000,
         2.4968,  1.5484,  3.4477,  3.1148,  1.6097,  8.9955,  2.7903,  5.6871,
         1.1277, 11.0084,  0.8419,  0.7490], device='cuda:2')

In [3]:
s = 0
c = 0

for i, (image, target) in enumerate(train_dl):
    #print(i)
    #if i==10:
    #    break
    image = image.to(device=device, dtype=torch.float)
    target = target.to(device=device, dtype=torch.float)
    s += target.sum(dim=0)
    c += 1

In [4]:
# raw with sampler
target.sum(dim=0)

tensor([0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 1., 0., 0., 0., 2., 0., 0.], device='cuda:2')

In [4]:
s/c * torch.from_numpy(class_weights_arr).to(device=device, dtype=torch.float)

tensor([ 7.1379,  4.0479,  3.3621,  6.2534,  7.3800,  6.2928,  9.0217,  4.0966,
         4.6807,  0.8745,  2.3483,  5.8655,  8.5893,  3.4607,  7.5424,  0.9793,
         5.2045,  4.6345,  3.9159,  5.9738,  0.3441,  6.9155,  3.9372,  3.8552,
         3.1645, 11.2610,  1.9500,  0.0000], device='cuda:2')

In [4]:
# not modify minor class
s/c * torch.from_numpy(class_weights_arr).to(device=device, dtype=torch.float)

tensor([6.8966, 2.5948, 3.8328, 5.5800, 7.8300, 4.8590, 9.0217, 4.0207, 4.0424,
        4.8097, 3.2876, 4.1276, 8.4648, 5.1910, 6.3400, 1.4690, 4.1369, 0.1655,
        4.7221, 5.2883, 3.7855, 6.5203, 3.6986, 3.4845, 1.8083, 8.4555, 2.2500,
        1.6014], device='cuda:2')

In [6]:
s

tensor([200.,  25.,  57.,  58.,  87.,  61.,  81.,  53.,  19.,  22.,  14.,  38.,
         68.,  39.,  58.,   6.,  31.,   1.,  41.,  54.,  22.,  99.,  31.,  47.,
         12., 217.,  15.,   6.], device='cuda:2')

In [5]:
s

tensor([207.,  39.,  50.,  65.,  82.,  79.,  81.,  54.,  22.,   4.,  10.,  54.,
         69.,  26.,  69.,   4.,  39.,  28.,  34.,  61.,   2., 105.,  33.,  52.,
         21., 289.,  13.,   0.], device='cuda:2')

In [76]:
s = target_arr.sum(axis=0)

In [77]:
dict((i,s[i]) for i in range(28)) 

{0: 22368,
 1: 3033,
 2: 10871,
 3: 3279,
 4: 5044,
 5: 5817,
 6: 3661,
 7: 9404,
 8: 207,
 9: 197,
 10: 182,
 11: 2173,
 12: 2199,
 13: 1430,
 14: 2651,
 15: 57,
 16: 1275,
 17: 446,
 18: 1877,
 19: 3634,
 20: 421,
 21: 13809,
 22: 2676,
 23: 10345,
 24: 424,
 25: 22210,
 26: 696,
 27: 127}

In [78]:
#mu in "create_class_weight" is a dampening parameter that could be tuned

import numpy as np
import math

def create_class_weight(labels_dict, mu=0.5):
    total = np.sum(list(labels_dict.values()))
    keys = list(labels_dict.keys())
    class_weight = dict()
    class_weight_log = dict()

    for key in keys:
        score = total / float(labels_dict[key])
        score_log = math.log(mu * total / float(labels_dict[key]))
        class_weight[key] = round(score, 2) if score > 1.0 else round(1.0, 2)
        class_weight_log[key] = round(score_log, 2) if score_log > 1.0 else round(1.0, 2)

    return class_weight, class_weight_log

# Class abundance for protein dataset
# labels_dict = {
#     0: 12885,
#     1: 1254,
#     2: 3621,
#     3: 1561,
#     4: 1858,
#     5: 2513,
#     6: 1008,
#     7: 2822,
#     8: 53,
#     9: 45,
#     10: 28,
#     11: 1093,
#     12: 688,
#     13: 537,
#     14: 1066,
#     15: 21,
#     16: 530,
#     17: 210,
#     18: 902,
#     19: 1482,
#     20: 172,
#     21: 3777,
#     22: 802,
#     23: 2965,
#     24: 322,
#     25: 8228,
#     26: 328,
#     27: 11
# }
labels_dict =\
{0: 22368,
 1: 3033,
 2: 10871,
 3: 3279,
 4: 5044,
 5: 5817,
 6: 3661,
 7: 9404,
 8: 207,
 9: 197,
 10: 182,
 11: 2173,
 12: 2199,
 13: 1430,
 14: 2651,
 15: 57,
 16: 1275,
 17: 446,
 18: 1877,
 19: 3634,
 20: 421,
 21: 13809,
 22: 2676,
 23: 10345,
 24: 424,
 25: 22210,
 26: 696,
 27: 127}

print('\nTrue class weights:')
print(create_class_weight(labels_dict)[0])
print('\nLog-dampened class weights:')
print(create_class_weight(labels_dict)[1])


True class weights:
{0: 5.83, 1: 43.03, 2: 12.01, 3: 39.8, 4: 25.87, 5: 22.44, 6: 35.65, 7: 13.88, 8: 630.5, 9: 662.5, 10: 717.1, 11: 60.06, 12: 59.35, 13: 91.27, 14: 49.23, 15: 2289.7, 16: 102.36, 17: 292.63, 18: 69.53, 19: 35.91, 20: 310.01, 21: 9.45, 22: 48.77, 23: 12.62, 24: 307.81, 25: 5.88, 26: 187.52, 27: 1027.66}

Log-dampened class weights:
{0: 1.07, 1: 3.07, 2: 1.79, 3: 2.99, 4: 2.56, 5: 2.42, 6: 2.88, 7: 1.94, 8: 5.75, 9: 5.8, 10: 5.88, 11: 3.4, 12: 3.39, 13: 3.82, 14: 3.2, 15: 7.04, 16: 3.94, 17: 4.99, 18: 3.55, 19: 2.89, 20: 5.04, 21: 1.55, 22: 3.19, 23: 1.84, 24: 5.04, 25: 1.08, 26: 4.54, 27: 6.24}


In [ ]:
class WeightedStratifiedSampler(Sampler):
    """Weighted Stratified Sampling
    Given class-weights of target classes in each batch.
    """
    def __init__(self, target):
        """
        Arguments
        ---------
        target : torch tensor
            a vector of class labels
        batch_size : integer
            batch_size
        """
        self.n_splits = int(target.size(0) / 32)
        self.target = target

    def gen_sample_array(self):
        try:
            from sklearn.model_selection import StratifiedShuffleSplit
        except:
            print('Need scikit-learn for this functionality')
        import numpy as np
        
        s = StratifiedShuffleSplit(n_splits=self.n_splits, test_size=0.5)
        X = th.randn(self.target.size(0),2).numpy()
        y = self.target.numpy()
        s.get_n_splits(X, y)

        train_index, test_index = next(s.split(X, y))
        return np.hstack([train_index, test_index])

    def __iter__(self):
        return iter(self.gen_sample_array())

    def __len__(self):
        return len(self.target)

## check external data and pick good images for training

In [30]:
from glob import glob

In [2]:
df = pd.read_csv('data/raw/external_data.csv')

In [52]:
## delete images with <4 channels (cannot use for training)
l = glob('data/raw/external_data_part3/*')
l = [_l.split('/')[-1].replace('_blue.png', '').replace('_red.png', '').replace('_green.png', '').replace('_yellow.png', '') 
     for _l in l]
vc = pd.value_counts(l)
complete_image_list = vc.loc[vc==4, ].index.tolist()
missing_channel_fname_list = vc.loc[vc<4, ].index.tolist()

In [38]:
l = [item.split('/')[-1] for item in glob('data/raw/bin/*/*')]

In [40]:
np.save('data/processed/downloaded_fname_list.npy', l)

In [55]:
len(missing_channel_fname_list), len(complete_image_list), len(l)

(0, 23998, 95992)

In [82]:
broken = []
for f in train_label.Id:
    for color in ['_red.png', '_green.png', '_blue.png', '_yellow.png']:
        try:
            img_file = glob('data/raw/*/'+f+color)[0]
        except:
            print(f)
            broken.append(f)

22076_615_D10_1
22076_615_D10_1
22076_615_D10_1
23370_182_B12_2
77752_1636_D3_32
77752_1636_D3_32
77752_1636_D3_32
64301_1169_A12_1
64301_1169_A12_1
54431_1179_H7_2
54431_1179_H7_2
54431_1179_H7_2
6104_7_G5_1
57384_1377_F4_3
57384_1377_F4_3
57384_1377_F4_3
41995_540_H5_2
41995_540_H5_2


In [85]:
broken

['22076_615_D10_1',
 '22076_615_D10_1',
 '22076_615_D10_1',
 '23370_182_B12_2',
 '77752_1636_D3_32',
 '77752_1636_D3_32',
 '77752_1636_D3_32',
 '64301_1169_A12_1',
 '64301_1169_A12_1',
 '54431_1179_H7_2',
 '54431_1179_H7_2',
 '54431_1179_H7_2',
 '6104_7_G5_1',
 '57384_1377_F4_3',
 '57384_1377_F4_3',
 '57384_1377_F4_3',
 '41995_540_H5_2',
 '41995_540_H5_2']

In [49]:
moved_fname_list = []
error_fname_list = []
for src in tqdm_notebook(glob('data/raw/bin/temp_upload_images*/*')):
    try:
        fname = src.split('/')[-1]
        dst = 'data/raw/external_data_part3/'+fname
        shutil.move(src, dst)
        moved_fname_list.append(src)
    except:
        error_fname_list.append(src)

In [55]:
y_train.mean(axis=0)

array([0.44444444, 0.03333333, 0.1       , 0.06666667, 0.04444444,
       0.13333333, 0.01111111, 0.05555556, 0.        , 0.01111111,
       0.01111111, 0.03333333, 0.01111111, 0.01111111, 0.04444444,
       0.01111111, 0.03333333, 0.        , 0.01111111, 0.07777778,
       0.01111111, 0.14444444, 0.04444444, 0.11111111, 0.        ,
       0.26666667, 0.02222222, 0.        ])

In [25]:
train_dl, val_dl = prepare_trainset(BATCH_SIZE, NUM_WORKERS, SEED, debug=False)

Count of trainset:  90
Count of validset:  10


In [26]:
x_train[0].shape

(4, 512, 512)

In [13]:
#fname_list = glob('data/raw/external_data_part3/*')
np.save('data/processed/external_images_list.npy', fname_list)

In [45]:
broken_files = []

for f in glob('data/processed/external_images_broken_list*'):
    l = np.load(f).tolist()
    broken_files.extend(l)
broken_files

['../raw/external_data_part3/23874_196_H4_1_yellow.png',
 '../raw/external_data_part3/16992_1606_H3_6_red.png',
 '../raw/external_data_part3/7979_22_G2_2_yellow.png']

In [9]:
# ## save trainset
# with open('data/processed/trainset_%d.pkl'%SEED, 'wb') as f:#trainset_aug_%d.pkl
#     pickle.dump([x_train, x_valid, y_train, y_valid, fname_train, fname_valid], f, protocol=4)

# x_train.shape, y_train.shape

# ## load trainset
# with open('data/processed/trainset_%d.pkl'%SEED, 'rb') as f:
#     x_train, x_valid, y_train, y_valid, fname_train, fname_valid = pickle.load(f)

# x_train.shape, y_train.shape

In [28]:
train_target = []
valid_target = []
full_target = []

for fname in fname_train:
    target = train_label.loc[train_label['Id']==fname, 'Target'].values[0]
    target = multi_hot(target)
    train_target.append(target)
train_target = np.array(train_target)

for fname in fname_valid:
    target = train_label.loc[train_label['Id']==fname, 'Target'].values[0]
    target = multi_hot(target)
    valid_target.append(target)
valid_target = np.array(valid_target)

for target in train_label.Target:
    target = multi_hot(target)
    full_target.append(target)
full_target = np.array(full_target)

In [22]:
train_target.mean(axis=0)

array([0.40200445, 0.0311804 , 0.11915367, 0.0545657 , 0.05790646,
       0.07572383, 0.0233853 , 0.09465479, 0.00334076, 0.00111359,
       0.00111359, 0.03006682, 0.0233853 , 0.00890869, 0.04231626,
       0.00111359, 0.00890869, 0.0077951 , 0.01781737, 0.05011136,
       0.00668151, 0.11804009, 0.02449889, 0.10356347, 0.01224944,
       0.25501114, 0.01002227, 0.00111359])

In [24]:
valid_target.mean(axis=0)

array([0.39215686, 0.02941176, 0.11764706, 0.05882353, 0.05882353,
       0.06862745, 0.01960784, 0.08823529, 0.        , 0.        ,
       0.        , 0.02941176, 0.01960784, 0.02941176, 0.03921569,
       0.        , 0.00980392, 0.        , 0.01960784, 0.04901961,
       0.00980392, 0.11764706, 0.02941176, 0.09803922, 0.00980392,
       0.25490196, 0.00980392, 0.        ])

In [40]:
# class sample count
#[round(i, 1) for i in full_target.mean(axis=0)*1000]

In [ ]:
def prepare_testset(BATCH_SIZE, NUM_WORKERS, SEED, debug=False):
    # read numpy format data
    with open('../data/processed/testset_%d.pkl'%SEED, 'rb') as f:
        mts_data_test, meta_data_test, y_test = pickle.load(f)
    y_test = y_test.astype(np.uint8)
    if debug:
        mts_data_test, meta_data_test, y_test = mts_data_test[:500], meta_data_test[:500], y_test[:500]
    print('Count of testset: ', mts_data_test.shape[0])

    # make pytorch.data.Dataset
    test_ds = PlasticDataSet(mts_data_test, meta_data_test, y_test)

    test_dl = DataLoader(
        test_ds,
        batch_size=BATCH_SIZE,
        shuffle=False,
        #sampler=StratifiedSampler(),
        num_workers=NUM_WORKERS,
    )
    
    return test_dl


